# Pràctica 3

In [2]:
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
train_es = conll2002.iob_sents('esp.train') # Train
dev_es = conll2002.iob_sents('esp.testa') # Dev
test_es =conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
dev_ned = conll2002.iob_sents('ned.testa') # Dev
test_ned =conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\sambr\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [36]:
train = conll2002.tagged_sents('esp.train')
test = conll2002.tagged_sents('esp.testb')
print(len(train))

8323


In [24]:
model = nltk.tag.CRFTagger()


In [ ]:
train_sample = train[0:500] #temps d'entrenament còmicament gran amb totes les dades: pillo un sample més petit
print(len(train_sample))

500


In [ ]:
model.train(train_sample, 'crfTaggerEs.mdl')


In [37]:
model.accuracy(test)

0.8961248132264762